In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-macosx_10_9_x86_64.whl size=1169137 sha256=a10d50aa7b12499704c830c11de8fe184d40d0f45d08bb887e8b12324278b284
  Stored in directory: /Users/harrychang/Library/Caches/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [33]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [34]:
# Load the datasets
anime_df = pd.read_csv("/Users/harrychang/Desktop/Y3S2/DSA4212/assignment2/assignment_2_data/assignment_2_anime.csv")
ratings_train_df = pd.read_csv("/Users/harrychang/Desktop/Y3S2/DSA4212/assignment2/assignment_2_data/assignment_2_ratings_train.csv")
ratings_test_df = pd.read_csv("/Users/harrychang/Desktop/Y3S2/DSA4212/assignment2/assignment_2_data/assignment_2_ratings_test.csv")

In [35]:
anime_df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [36]:
ratings_train_df

,user_id,anime_id,rating
0,20170,10794,6
1,24592,21995,5
2,18358,7054,10
3,59267,488,7
4,69313,30544,4
...,...,...,...
4436063,32872,15061,7
4436064,66206,20507,8
4436065,46386,834,7
4436066,10497,9159,7


In [37]:
ratings_test_df

,user_id,anime_id,rating
0,44017,13161,4
1,14307,14993,7
2,55155,268,9
3,63515,2889,9
4,54059,2581,7
...,...,...,...
1901168,56619,31483,10
1901169,24755,1210,8
1901170,17346,15225,6
1901171,28874,5041,8


In [38]:
anime_df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [46]:
# Preprocess the anime dataset
anime_df = anime_df.dropna()
anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
features    0
dtype: int64

In [47]:
# Combine anime features into a single string
anime_df['features'] = anime_df['genre'] + " " + anime_df['type']

<ipython-input-47-bca57b416f83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df['features'] = anime_df['genre'] + " " + anime_df['type']


In [48]:
# Create a TF-IDF matrix for the features
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(anime_df['features'])

In [49]:
# Perform K-means clustering using the TF-IDF matrix
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
anime_df['cluster'] = kmeans.fit_predict(tfidf_matrix)

/Users/harrychang/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-49-e0015e4b2ad0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df['cluster'] = kmeans.fit_predict(tfidf_matrix)


In [52]:
def predict_rating(user_id, anime_id):
    try:
    # Get the cluster of the given anime_id
        cluster = anime_df[anime_df['anime_id'] == anime_id]['cluster'].values[0]

    # Filter anime belonging to the same cluster
        same_cluster_anime = anime_df[anime_df['cluster'] == cluster]

    # Get the user's ratings for the same cluster anime
        user_ratings = ratings_train_df[ratings_train_df['user_id'] == user_id]
        user_ratings_same_cluster = user_ratings[user_ratings['anime_id'].isin(same_cluster_anime['anime_id'])]

    # If the user has not rated any anime in the same cluster, return the user's mean rating
        if user_ratings_same_cluster.empty:
            return user_ratings['rating'].mean()

    # Predict the rating based on the user's mean rating for the same cluster anime
        return user_ratings_same_cluster['rating'].mean()

    except IndexError:
        # Handle the IndexError here, e.g., return a default rating or print a warning
        print(f"IndexError encountered for user_id: {user_id} and anime_id: {anime_id}")
        return -1  # Return a default rating or any appropriate value

In [53]:
# Calculate the predicted ratings for the test dataset
ratings_test_df['predicted_rating'] = ratings_test_df.apply(
    lambda x: predict_rating(x['user_id'], x['anime_id']), axis=1)

IndexError encountered for user_id: 15221 and anime_id: 29765
IndexError encountered for user_id: 12431 and anime_id: 29922
IndexError encountered for user_id: 51693 and anime_id: 19219
IndexError encountered for user_id: 13954 and anime_id: 31831
IndexError encountered for user_id: 13954 and anime_id: 29957
IndexError encountered for user_id: 13954 and anime_id: 31507
IndexError encountered for user_id: 34240 and anime_id: 30913
IndexError encountered for user_id: 65155 and anime_id: 28647
IndexError encountered for user_id: 64880 and anime_id: 28655
IndexError encountered for user_id: 37442 and anime_id: 30913
IndexError encountered for user_id: 12431 and anime_id: 31831
IndexError encountered for user_id: 73188 and anime_id: 19219
IndexError encountered for user_id: 71038 and anime_id: 29629
IndexError encountered for user_id: 33787 and anime_id: 31834
IndexError encountered for user_id: 71038 and anime_id: 29655
IndexError encountered for user_id: 33787 and anime_id: 31831
IndexErr

In [28]:
ratings_train_df.isnull().values.any()

False

In [29]:
ratings_test_df.isnull().values.any()

False

In [54]:
anime_df

,anime_id,name,genre,type,episodes,rating,members,features,cluster
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"Drama, Romance, School, Supernatural Movie",31
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"Action, Adventure, Drama, Fantasy, Magic, Mili...",8
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"Action, Comedy, Historical, Parody, Samurai, S...",57
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"Sci-Fi, Thriller TV",93
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"Action, Comedy, Historical, Parody, Samurai, S...",57
...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,Hentai OVA,3
12290,5543,Under World,Hentai,OVA,1,4.28,183,Hentai OVA,3
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,Hentai OVA,3
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,Hentai OVA,3


In [57]:
# Fill NaN values in 'rating' column with the mean rating
mean_rating = ratings_test_df['rating'].mean()
ratings_test_df['rating'].fillna(mean_rating, inplace=True)

# Fill NaN values in 'predicted_rating' column with the mean predicted rating
mean_predicted_rating = ratings_test_df['predicted_rating'].mean()
ratings_test_df['predicted_rating'].fillna(mean_predicted_rating, inplace=True)

# Calculate the MSE for clustering
mse = mean_squared_error(ratings_test_df['rating'], ratings_test_df['predicted_rating'])
print("Clustering MSE:", mse)

Clustering MSE: 2.054134889019177
